In [1]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import jax
import jax.numpy as jnp
import sys
sys.path.append('..')
import analysis
import datatypes
from input_pipeline import ase_atoms_to_jraph_graph, generate_fragments
from qm9 import load_qm9

2023-02-27 12:23:54.169595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 12:23:55.794771: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-27 12:23:55.794889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-27 12:23:55.794901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
config, best_state, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace1')

/Users/ameyad/Documents/qm9_data_tf/


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ameyad/Documents/qm9_data_tf/'

In [5]:
atoms = load_qm9("qm9_data")[1]

In [51]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])  # is there a way to get just the number from a jax array, not a (1,) slice?
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [8]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

graph = ase_atoms_to_jraph_graph(atoms, atomic_numbers, cutoff)
frags = generate_fragments(key, graph, len(atomic_numbers), epsilon)
frags = list(frags)

In [63]:
frag = frags[3]
frag_atoms = Atoms(positions=frag.nodes.positions, numbers=get_numbers(frag.nodes.species))
view(frag_atoms, viewer='x3d')

In [ ]:
# save to image files instead:
for i, frag_to_write in enumerate(frags):
    write(f'image{i}.png', Atoms(positions=frag_to_write.nodes.positions, numbers=get_numbers(frag_to_write.nodes.species)))